In [ ]:
import json
import requests # Need this for making HTTP requests
import os
from typing import Annotated, Optional, List, Callable, Dict, Any
from function_schema import get_function_schema # Assuming function_schema.py exists with get_function_schema

# --- Tool Definitions (Provided by User) ---
def create_calendar_event(
    summary: Annotated[str, "Title of the event to be added (default: 'New Event')"],
    start_time: Annotated[str, "Start date and time of the event (format: 'yyyy-MM-dd HH:mm')"],
    end_time: Annotated[str, "End date and time of the event (format: 'yyyy-MM-dd HH:mm')"]
) -> None:
    """Creates a new calendar event."""
    pass

def fetch_calendar_events(
    start_date: Annotated[str, "Start date of the search range (format: yyyy-MM-dd)"],
    end_date: Annotated[str, "End date of the search range (format: yyyy-MM-dd)"]
) -> str:
    """
    Retrieves calendar events within a specified date range.
    Requires authorization first. If not authorized, should call authorize_calendar_access.
    Returns a JSON string representing the events or an error message.
    """
    pass

def authorize_calendar_access() -> None:
    """
    Initiates the authorization process for calendar access.
    Must be called first before using calendar-related tools like fetch_calendar_events or create_calendar_event if not already authorized.
    """
    pass

def web_search(
    query: Annotated[str, "The query to search for on the web."]
) -> str:
    """
    Searches the web (DuckDuckGo) for the given query.
    Returns a JSON string containing search results.
    """
    pass

# --- Tool Setup ---
tools = [
    create_calendar_event,
    fetch_calendar_events,
    authorize_calendar_access,
    web_search,
]

# Generate schemas and store them in a dictionary keyed by function name
tool_schemas = {tool.__name__: get_function_schema(tool) for tool in tools}

# --- LLM Configuration (Replace with your actual details) ---
FRIENDLI_API_TOKEN = os.environ.get("FRIENDLI_TOKEN", "YOUR_FRIENDLI_API_TOKEN") # Use environment variable or replace placeholder
FRIENDLI_API_URL = f"https://api.friendli.ai/serverless/v1/chat/completions"

# --- Virtual Tool Executor ---

def simulate_tool_call(
    function_name: str,
    args: Dict[str, Any],
    tool_schemas_dict: Dict[str, Dict] = tool_schemas, # Use the globally defined schemas by default
    api_token: str = FRIENDLI_API_TOKEN,
    endpoint_id: str = "meta-llama-3.3-70b-instruct",
    api_url: str = FRIENDLI_API_URL
) -> str:
    """
    Simulates the execution of a tool using an LLM.

    Args:
        function_name: The name of the function to simulate.
        args: A dictionary of arguments for the function call.
        tool_schemas_dict: A dictionary mapping function names to their schemas.
        api_token: The API token for the LLM service.
        endpoint_id: The specific model endpoint ID for the LLM service.
        api_url: The API endpoint URL for the LLM service.

    Returns:
        A JSON string representing the simulated output of the tool,
        or an error JSON string if the simulation fails.
    """
    print(f"--- Simulating call to {function_name} with args: {args} ---")

    if function_name not in tool_schemas_dict:
        print(f"Error: Function '{function_name}' not found in tool schemas.")
        return json.dumps({"error": f"Function '{function_name}' not found."})

    if not api_token or api_token == "YOUR_FRIENDLI_API_TOKEN":
         print("Error: Friendli API token not configured.")
         return json.dumps({"error": "LLM API token not configured."})

    if not endpoint_id or endpoint_id == "YOUR_ENDPOINT_ID":
            print("Error: Friendli endpoint ID not configured.")
            return json.dumps({"error": "LLM endpoint ID not configured."})


    schema = tool_schemas_dict[function_name]
    function_description = schema.get("description", "No description provided.")

    # Construct the prompt for the LLM
    prompt = f"""You are an expert function simulator. Based on the following function description and the provided arguments, simulate the execution of this function call.

Function Name: {function_name}

Function Description: {function_description}

Function Schema:
{json.dumps(schema, indent=2)}

Arguments Provided:
{json.dumps(args, indent=2)}

Current Date/Time: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')} (Assume this is the time of execution)

Task:
Generate a plausible JSON response string that represents what the function '{function_name}' would return if it were actually executed with the given arguments.
- Consider the function's description (e.g., does it fetch data, create something, authorize, search?).
- Consider the argument values (e.g., dates, search terms).
- If the function description mentions potential errors (like needing authorization for 'fetch_calendar_events'), sometimes simulate those error responses.
- If the function returns nothing on success (like 'create_calendar_event' or 'authorize_calendar_access'), return a JSON indicating success, like '{{"status": "success"}}' or an empty JSON object '{{}}'.
- For functions returning data (like 'fetch_calendar_events' or 'web_search'), generate realistic-looking example data formatted as a JSON string.
- Ensure your entire output is ONLY the JSON string, without any introductory text, explanations, or markdown formatting like ```json ... ```. Just the raw JSON string.
"""

    # Prepare the API request data
    request_data = {
        "model": endpoint_id,
        "messages": [
            {"role": "system", "content": "You are an expert function simulator outputting only JSON strings."},
            {"role": "user", "content": prompt}
        ],
        "max_tokens": 500, # Adjust as needed
        "temperature": 0.5 # Adjust for creativity vs determinism
    }

    headers = {
        "Authorization": f"Bearer {api_token}",
        "Content-Type": "application/json"
    }

    # Make the API call
    try:
        print(f"Sending request to LLM: {api_url}")
        response = requests.post(api_url, headers=headers, json=request_data, timeout=30) # Added timeout
        response.raise_for_status() # Raise an exception for bad status codes (4xx or 5xx)

        response_json = response.json()

        # Extract the content from the response
        # Structure might vary slightly based on API; adjust if necessary
        if "choices" in response_json and len(response_json["choices"]) > 0:
            message = response_json["choices"][0].get("message", {})
            simulated_output = message.get("content", "").strip()

            # Basic validation: Check if it looks like JSON
            if (simulated_output.startswith('{') and simulated_output.endswith('}')) or \
               (simulated_output.startswith('[') and simulated_output.endswith(']')) or \
               (simulated_output == 'null'):
                print(f"LLM simulation successful. Raw output:\n{simulated_output}")
                # Optional: Validate if it's truly valid JSON
                try:
                    json.loads(simulated_output)
                    return simulated_output
                except json.JSONDecodeError:
                    print(f"Warning: LLM output is not valid JSON: {simulated_output}")
                    # Fallback: Return an error JSON or the invalid string itself
                    return json.dumps({"error": "LLM returned invalid JSON", "raw_output": simulated_output})
            else:
                 print(f"Warning: LLM output doesn't look like JSON: {simulated_output}")
                 # Fallback: Return an error JSON or the raw string
                 return json.dumps({"error": "LLM output doesn't look like JSON", "raw_output": simulated_output})
        else:
            print("Error: Unexpected LLM response format.")
            print("Response:", response_json)
            return json.dumps({"error": "Unexpected LLM response format", "details": response_json})

    except requests.exceptions.RequestException as e:
        print(f"Error calling LLM API: {e}")
        error_details = str(e)
        if hasattr(e, 'response') and e.response is not None:
             try:
                 error_details = e.response.json()
             except json.JSONDecodeError:
                 error_details = e.response.text
        return json.dumps({"error": "Failed to call LLM API", "details": error_details})
    except Exception as e:
        print(f"An unexpected error occurred during simulation: {e}")
        return json.dumps({"error": "An unexpected error occurred", "details": str(e)})


from datetime import datetime, timedelta






--- Example 1: Create Calendar Event ---
--- Simulating call to create_calendar_event with args: {'summary': 'Team Meeting', 'start_time': '2025-05-05 10:00', 'end_time': '2025-05-05 11:00'} ---
Sending request to LLM: https://api.friendli.ai/serverless/v1/chat/completions
LLM simulation successful. Raw output:
{"status": "success", "event_id": "EV123456789", "summary": "Team Meeting", "start_time": "2025-05-05 10:00", "end_time": "2025-05-05 11:00"}
Simulated Result (JSON String): {"status": "success", "event_id": "EV123456789", "summary": "Team Meeting", "start_time": "2025-05-05 10:00", "end_time": "2025-05-05 11:00"}

--- Example 2: Fetch Calendar Events (Success Scenario) ---
--- Simulating call to fetch_calendar_events with args: {'start_date': '2025-05-04', 'end_date': '2025-05-11'} ---
Sending request to LLM: https://api.friendli.ai/serverless/v1/chat/completions
LLM simulation successful. Raw output:
{"events": [
  {
    "id": 1,
    "title": "Team Meeting",
    "start": "202

In [5]:
print("\n--- Example 1: Create Calendar Event ---")
result1 = simulate_tool_call(
    function_name="create_calendar_event",
    args={
        "summary": "Team Meeting",
        "start_time": (datetime.now() + timedelta(days=1)).strftime("%Y-%m-%d 10:00"),
        "end_time": (datetime.now() + timedelta(days=1)).strftime("%Y-%m-%d 11:00")
    }
)
print("Simulated Result (JSON String):", result1)


--- Example 1: Create Calendar Event ---
--- Simulating call to create_calendar_event with args: {'summary': 'Team Meeting', 'start_time': '2025-05-05 10:00', 'end_time': '2025-05-05 11:00'} ---
Sending request to LLM: https://api.friendli.ai/serverless/v1/chat/completions
LLM simulation successful. Raw output:
{"status": "success", "event_id": "EVNT-20250505-001", "summary": "Team Meeting", "start_time": "2025-05-05 10:00", "end_time": "2025-05-05 11:00"}
Simulated Result (JSON String): {"status": "success", "event_id": "EVNT-20250505-001", "summary": "Team Meeting", "start_time": "2025-05-05 10:00", "end_time": "2025-05-05 11:00"}
